<a href="https://colab.research.google.com/github/bkyappworks/image-repository/blob/main/Startup_Investments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Research Scope


- Quick over view of the startups worldwide
  - Number of Startups in terms of time worldwide
    - 2000 and 2008 peak(delayed) -  The dot-com bubble in the United States and financial crisis
    - huge drop started from 2011
  - Software is the leading market in startups
  - Distribution of status over all startups (Do we need this?)
  - USA dominants the startups worldwide
 
- Dive into USA startups
  - time: 
    - general patterns: Most startups are founded in January
    - dramatically drop in 2011 & Obama's "Startup America" -> DID
    - biotech vs. software trend
    - market trend before and after financial crisis (Obama care and policy) 
  - state: 
    - Most popular state founded in USA (CA,NY,MA,TX and FL are where most startups are founded)
    - what are the most popular market in those states?
    - Zoom in each popular and find the average funding in terms of status -> top market with num is not the same as the top market with avg fundings -> typical markets in each state
   e.g. NY: Media and Newspaper


## Data loading and pre-processing

Import packages

In [1]:
pip install pandas_bokeh

In [2]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_bokeh

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Cathy's version
# DF = pd.read_csv('/content/drive/MyDrive/project-1 team14/investments_VC.csv',encoding='latin1')
#Becky's version
DF = pd.read_csv('/content/drive/MyDrive/FALL 2021/95-885 Data Science and Big Data/project-1 team14/Final Deliverables/investments_VC.csv',encoding='latin1')

Check the number of rows and columns in the dataset

In [7]:
print('The dataset includes {} rows.'.format(len(DF)))
print('The dataset inlcudes {} columns.'.format(len(DF.columns)))

The dataset includes 54294 rows.
The dataset inlcudes 39 columns.


Check the missing values in the dataset

In [8]:
DF.isna().sum()

permalink                4856
name                     4857
homepage_url             8305
category_list            8817
 market                  8824
 funding_total_usd       4856
status                   6170
country_code            10129
state_code              24133
region                  10129
city                    10972
funding_rounds           4856
founded_at              15740
founded_month           15812
founded_quarter         15812
founded_year            15812
first_funding_at         4856
last_funding_at          4856
seed                     4856
venture                  4856
equity_crowdfunding      4856
undisclosed              4856
convertible_note         4856
debt_financing           4856
angel                    4856
grant                    4856
private_equity           4856
post_ipo_equity          4856
post_ipo_debt            4856
secondary_market         4856
product_crowdfunding     4856
round_A                  4856
round_B                  4856
round_C   

## Data Cleaning

Make a copy of the original dataset and start the data clearning

In [9]:
df = DF.copy()

Deal with missing values

In [10]:
# drop rows containing all NAs
df = df.dropna(how = 'all')
df.isna().sum()

permalink                   0
name                        1
homepage_url             3449
category_list            3961
 market                  3968
 funding_total_usd          0
status                   1314
country_code             5273
state_code              19277
region                   5273
city                     6116
funding_rounds              0
founded_at              10884
founded_month           10956
founded_quarter         10956
founded_year            10956
first_funding_at            0
last_funding_at             0
seed                        0
venture                     0
equity_crowdfunding         0
undisclosed                 0
convertible_note            0
debt_financing              0
angel                       0
grant                       0
private_equity              0
post_ipo_equity             0
post_ipo_debt               0
secondary_market            0
product_crowdfunding        0
round_A                     0
round_B                     0
round_C   

Format the columns

In [11]:
# make sure the columns don't contain any space 
df.columns = df.columns.str.replace(' ','')
df['market'] = df['market'].str.replace(' ', '')

In [12]:
# replace the missing total funding with 0 
df['funding_total_usd'] = df['funding_total_usd'].str.replace(',', '')
df['funding_total_usd'] = df['funding_total_usd'].str.replace(' ', '')
df['funding_total_usd'] = df['funding_total_usd'].str.replace('-', '0')
df['funding_total_usd'] = df['funding_total_usd'].astype(float)

Drop duplicates

In [13]:
df = df.drop_duplicates()

Drop unnecessary columns

In [14]:
unne_col = ['permalink', 'homepage_url', 'city', 'founded_quarter']
df = df.drop(columns= unne_col)

Remove illogical rows

In [15]:
# the first funding date is earlier than the founded date
df = df[df['founded_at'] <= df['first_funding_at']]


Create new columns

In [16]:
df['lasting_year'] = 2014 - df['founded_year']
y = pd.get_dummies(df['status'], prefix='Status')
df = pd.concat([df,y], axis=1)
df['if_ipo'] = np.where((df['post_ipo_debt'] != 0) | (df['post_ipo_equity'] != 0) | (df['secondary_market'] > 0), 1, 0)
df['month'] = pd.to_datetime(df['founded_month'], errors='coerce')
df['month'] = df['month'].dt.month
df['if_jan'] = df['month'].apply(lambda x: 1 if x == 1 else 0)
df['found_fund_diff'] = (pd.to_datetime(df['first_funding_at'], errors = 'coerce') - pd.to_datetime(df['founded_at'], errors = 'coerce')).astype('timedelta64[D]')
df['fir_end_diff'] = (pd.to_datetime(df['last_funding_at'], errors = 'coerce') - pd.to_datetime(df['first_funding_at'], errors = 'coerce')).astype('timedelta64[D]')
df['founded_year_range'] = pd.cut(df['founded_year'], bins=[1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020], right=False)

Check the number of rows and columns in the dataset after cleaning

In [17]:
print('The dataset includes {} rows after data cleaning.'.format(len(df)))
print('The dataset inlcudes {} columns after data cleaning.'.format(len(df.columns)))

The dataset includes 35809 rows after data cleaning.
The dataset inlcudes 45 columns after data cleaning.


In [18]:
df.columns

Index(['name', 'category_list', 'market', 'funding_total_usd', 'status',
       'country_code', 'state_code', 'region', 'funding_rounds', 'founded_at',
       'founded_month', 'founded_year', 'first_funding_at', 'last_funding_at',
       'seed', 'venture', 'equity_crowdfunding', 'undisclosed',
       'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H',
       'lasting_year', 'Status_acquired', 'Status_closed', 'Status_operating',
       'if_ipo', 'month', 'if_jan', 'found_fund_diff', 'fir_end_diff',
       'founded_year_range'],
      dtype='object')

## Summary Statistics

1. Construct a bar chart that displays the number of startups in terms of time.


In [19]:
df_year = df[df['founded_year'] < 2013]
df_year = df_year.groupby(['founded_year'])['name'].count()
df_year = df_year.reset_index()
df_year.rename({'founded_year': 'Founded_Year', 'name': 'Count_of_Records'}, axis=1, inplace=True)

year_plt = alt.Chart(df_year, title = 'Number of startups reaches the peak in 2012 worldwide').mark_line().encode(
           alt.X('Founded_Year:N', title = 'Founded Year'),
           alt.Y('Count_of_Records:Q', title = 'Count of Records'),
           tooltip = ['Founded_Year', 'Count_of_Records']
           ).properties(width=800, height=300).interactive()
year_plt


alt.Chart(...)

Based on the visualization above,  we find two small drops from 1902 to 2012, occurring in 2000 and 2007. We think that the 2000 recession and 2008 financial crisis are the main reasons that caused the decline or lag decline of startups in corresponding years. 


In [20]:
df_name = df.copy()
df_name = df_name[df_name['name'].notna()]
df_name['if_com'] = df_name['name'].str.contains('\.com')
df_name['if_com'] = df_name['if_com'].astype(int)

group_count = df_name.groupby(['founded_year_range'])['name'].count()
group_count.index = group_count.index.astype(str)
group_count = group_count.reset_index()

group_sum = df_name.groupby(['founded_year_range'])['if_com'].sum()
group_sum.index = group_sum.index.astype(str)
group_sum = group_sum.reset_index()

group_count['Company_with_com'] = group_sum['if_com']
group_count['Percentage'] = round(group_sum['if_com']/group_count['name']*100,2)

group_count.rename({'founded_year_range': 'Year_Range', 'name': 'Count_of_Records'}, axis=1, inplace=True)

sep = alt.Chart(group_count, title= 'The shift to "Information Age" started from 1990').mark_bar().encode(
    alt.X('Year_Range:N'),
    alt.Y('Percentage:Q', title = 'Percentage'),
    tooltip = ['Year_Range', 'Percentage', 'Count_of_Records']
).interactive()

bar1 =  alt.Chart(group_count).mark_line(color='#5276A7').encode(
    alt.X('Year_Range:N'),
    alt.Y('Count_of_Records', title = 'Count of Records'),
    tooltip = ['Year_Range', 'Count_of_Records']
).interactive()
bar2 =  alt.Chart(group_count).mark_line(color='#A40066').encode(
    alt.X('Year_Range:N'),
    alt.Y('Company_with_com', title = 'Count of .com'),
    tooltip = ['Year_Range', 'Company_with_com']
).interactive()
together = alt.layer(bar1, bar2)

In [21]:
sep | together


alt.HConcatChart(...)

The release of Mosiac in 1993 and following web browsers gave computer users access to the World Wide Web. Based on [wikipedia](https://en.wikipedia.org/wiki/Dot-com_bubble), the percentage of households in the United States owning computers increased from 15% to 35% as computer ownership progressed from a luxury to a necessity from 1990 to 1997. At the same time, many investors were eager to invest in any dot-com company, especially if it had one of the Internet-related prefixes or a ".com" suffix in its name.

According to the visualization, we find that the number of startups containing ".com" as a suffix in its name has increased dramatically from 1990 through 2000, parallel with the shift to "Information Age." The percentage of startups that contained ".com" as a suffix in its name in 1910-1920 is much more significant than other periods. The reason behind it is that one company among a total of 19 startups contained ".com" as a suffix in its name, which makes the percentage more remarkable with a smaller denominator. 

2. Construct a bar chart that displays the top 20 markets for startups

In [22]:
df_market = df.groupby(['market'])['name'].count()
df_market = df_market.reset_index().sort_values(by = 'name',ascending = False)[:20]
df_market.rename({'market': 'Market', 'name': 'Count_of_Records'}, axis=1, inplace=True)

alt.Chart(df_market, title= 'Software is the leading market in all startups').mark_bar().encode(
    alt.X('Market:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Count_of_Records:Q', title = 'Count of Records'),
    alt.Color('Market:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['Market', 'Count_of_Records']
).interactive()
#present

alt.Chart(...)

Most markets included in this top list are related to technology such as software, biotechnology, mobile, E-commerce, etc. In other words, the technology-related market is the most popular one that founders work for. Currently, we are entering the [fifth generation](https://www.mhconsults.com/blog/what-are-the-five-generations-of-computer-technology/) of technology, from information technology to artificial technology. **New startups should keep innovating and exploring how to adapt artificial technology toward individual business needs.**

3. Constract a bar chart displays the top 15 countires for startups

In [23]:
df_country = df.groupby(['country_code'])['name'].count()
df_country = df_country.reset_index().sort_values(by = 'name',ascending = False)[:15]
df_country.rename({'country_code': 'Country', 'name': 'Count_of_Records'}, axis=1, inplace=True)
top_10_country = list(df_country['Country'])

alt.Chart(df_country, title = "The number of USA's startups is dominant worldwide").mark_bar().encode(
    alt.X('Country:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Count_of_Records:Q', title = 'Count of Records'),
    alt.Color('Country:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['Country', 'Count_of_Records']
).interactive()
#present

alt.Chart(...)

The United States dominates the startups worldwide. Based on the visualization, we find that number of startups in the United States is 12 times the United Kingdom (2nd place for the top countries in terms of the total number of startups) from 1902 to 2014.

Since we have more data in the United States, we want to dig deeper about the startups in the United States and give suggestions for investors and entrepreneurs who want to start their startups in the United States now.

4. Build a bar chart that depicts the status distribution of startups

In [24]:
df_status = df.groupby(['status'])['name'].count()
df_status = df_status.reset_index()

fig = px.pie(df_status, values='name', names='status', 
             color_discrete_sequence=px.colors.sequential.RdBu, title='Startups that are still opearting take 87.1% of all startups')
fig.show()

## Deeper Exploration for Startups in the United States

### Quick exploration of startups in the United States

In [25]:
# due to a data integrity problems, we cut the data in 2013 and 2014
df_usa = df[(df['country_code'] == 'USA') & (df['founded_year'] < 2013)]


- Geographical distribution of startups in the United States

In [26]:
df_usa_state = df_usa.groupby(['state_code'])['name'].count()
df_usa_state = df_usa_state.reset_index().sort_values(by = 'name',ascending = False)[:20]
df_usa_state.rename({'state_code': 'State', 'name': 'Count_of_Records'}, axis=1, inplace=True)

alt.Chart(df_usa_state, title = "Number of startups in California dominates the startups in the United States").mark_bar().encode(
    alt.X('State:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Count_of_Records:Q', title = 'Count of Records'),
    alt.Color('State:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['State', 'Count_of_Records']
).interactive()

alt.Chart(...)

- Market distribution of startups in the United States

In [27]:
df_usa_state = df_usa.groupby(['market'])['name'].count()
df_usa_state = df_usa_state.reset_index().sort_values(by = 'name',ascending = False)[:20]
df_usa_state.rename({'market': 'Market', 'name': 'Count_of_Records'}, axis=1, inplace=True)

alt.Chart(df_usa_state, title = "Number of startups in California dominates the startups in the United States").mark_bar().encode(
    alt.X('Market:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Count_of_Records:Q', title = 'Count of Records'),
    alt.Color('Market:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['Market', 'Count_of_Records']
).interactive()

alt.Chart(...)

- Correlation plot

In [28]:
# correlation
df_usa_corr = df_usa[['funding_total_usd', 'funding_rounds', 'seed', 'venture', 'equity_crowdfunding', 'undisclosed',
       'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H',
       'lasting_year', 'Status_acquired', 'Status_closed', 'Status_operating',
       'if_ipo', 'if_jan', 'found_fund_diff', 'fir_end_diff']]

df_usa_corr = df_usa_corr.corr().stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable 1', 'level_1': 'variable 2'})
df_usa_corr['correlation_label'] = df_usa_corr['correlation'].map('{:.2f}'.format)

base = alt.Chart(df_usa_corr).encode(
    x='variable 2:O',
    y='variable 1:O'    
).properties(width = 800, height = 800)

text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

cor_plot = base.mark_rect().encode(
    color='correlation:Q'
)

cor_plot + text

alt.LayerChart(...)

### Findings 1: Unicorn startups

In [31]:
unicorn = pd.read_excel("/content/drive/MyDrive/FALL 2021/95-885 Data Science and Big Data/project-1 team14/Final Deliverables/uniorn_companies.xlsx")
unicorn_list = unicorn['Company'].tolist()
df_unicorn = df[df['name'].isin(unicorn_list)].dropna(subset = ['market'])
len(unicorn_list)/df.dropna(subset = ['market']).shape[0] #0.017962308598351
df.dropna(subset = ['market']).shape[0] #33960

33960

In [32]:
df_unicorn_m = df[df['name'].isin(unicorn_list)].groupby('market')['name'].count().reset_index().sort_values(by = 'name',ascending = False)
unicorn_markets = df_unicorn_m['market'].tolist()

In [33]:
df_unicorn.groupby('country_code')['name'].count().reset_index().sort_values(by = 'name',ascending = False).head()

,country_code,name
17,USA,89
4,CHN,8
8,GBR,6
11,ISR,6
10,IND,5


In [34]:
unicorn_year = df_unicorn.groupby('founded_year')['name'].count().reset_index().sort_values(by = 'name',ascending = False)
unicorn_year.head()

,founded_year,name
16,2012.0,25
15,2011.0,22
17,2013.0,16
13,2009.0,16
14,2010.0,15


In [35]:
def unicorn_by_year(df,start,end):
  df_subset = df[(df['founded_year']>start)&(df['founded_year']<end)]
  df_subset = df_subset.groupby(['founded_year','market'])['name'].count().reset_index()
  result = alt.Chart(df_subset).mark_bar().encode(
      alt.X('founded_year:N', axis=alt.Axis(labelAngle=0)),
      alt.Y('name:Q'),
      alt.Color('market:N'),
      tooltip=['market','founded_year','name']
  ).properties(title="Number of unicorns during "+str(start)+" - "+str(end),width=1000,height=380).interactive()

  return result
unicorn_by_year(df_unicorn,1990, 2011)

alt.Chart(...)

In [36]:
df_unicorn['new_market'] = df_unicorn['market'].apply(lambda x : "Others" if x not in unicorn_markets[:10] else x)
df_count = df_unicorn.groupby(['new_market'])['name'].count().reset_index()
fig = px.pie(df_count, values='name', names='new_market', 
             color_discrete_sequence=px.colors.sequential.RdBu, title=' Almost 12% of unicorns are in Software,E-commerce and Mobile markets')
fig.show()

In [37]:
un_country = df_unicorn.groupby('country_code').count()['name'].reset_index().sort_values(by = "name",ascending = False)['country_code'][:10].tolist()
df_unicorn['new_country'] = df_unicorn['country_code'].apply(lambda x : "Others" if x not in un_country else x)
df_uc_country = df_unicorn.groupby(['new_country'])['name'].count().reset_index()
df_uc_country.rename({'name': 'count'}, axis=1, inplace=True)
df_uc_country.sort_values(by = 'count',ascending = False).head()
fig = px.pie(df_uc_country, values='count', names='new_country', 
             color_discrete_sequence=px.colors.sequential.RdBu, title=' Over 60% of unicorns are in USA, followed by China, UK, Israel and India')
fig.show()

In [38]:
def unicorn_by_country(df,start,end):
  df_subset = df[(df['founded_year']>start)&(df['founded_year']<end)]
  df_subset = df_subset.groupby(['founded_year','country_code'])['name'].count().reset_index().sort_values(by = "name",ascending = False)
  df_subset.rename({'name': 'count'}, axis=1, inplace=True)
  result = alt.Chart(df_subset).mark_bar().encode(
      alt.X('founded_year:N', axis=alt.Axis(labelAngle=0)),
      alt.Y('count:Q'),
      alt.Color('country_code:N'),
      tooltip=['country_code','founded_year','count']
  ).properties(title="Number of unicorns during "+str(start)+" - "+str(end),width=1000,height=380).interactive()

  return result
unicorn_by_country(df_unicorn,1990, 2011)

alt.Chart(...)

In [39]:
topmarkets = df_unicorn.groupby('market')['name'].count().reset_index().sort_values(by = 'market')
allmarkets = df[df['market'].isin(topmarkets['market'].tolist())].groupby('market')['name'].count().reset_index().sort_values(by = 'market')
topmarkets['successful rate'] = round(topmarkets['name']/allmarkets['name'],3)
topmarkets.head()

,market,name,successful rate
0,3DPrinting,1,0.143
1,Analytics,4,0.008
2,Apps,2,0.011
3,Biotechnology,3,0.001
4,Bitcoin,1,0.027


In [40]:
top15 = topmarkets.sort_values(by = 'successful rate',ascending = False)[:15]
top15.rename({'name': 'count'}, axis=1, inplace=True)
top15.head()

,market,count,successful rate
10,DataPrivacy,1,0.500
54,SharedServices,1,0.333
63,Taxis,1,0.250
35,MobileEnterprise,1,0.250
22,Geospatial,1,0.250


In [41]:
top15f = topmarkets[topmarkets['name'] >= 3].sort_values(by = 'successful rate',ascending = False)[:15]
top15f.rename({'name': 'count'}, axis=1, inplace=True)
top15f.head()

,market,count,successful rate
46,Restaurants,3,0.056
31,Marketplaces,4,0.024
48,SaaS,3,0.014
12,E-Commerce,12,0.009
14,EnterpriseSoftware,9,0.009


In [42]:
import altair as alt
def successfulrate(source):
  result = alt.Chart(source).mark_circle(
      opacity=0.8,
      stroke='black',
      strokeWidth=1
  ).encode(
      alt.X('successful rate:O', axis=alt.Axis(labelAngle=0)),
      alt.Y('market:N'),
      alt.Size('count:Q',
          scale=alt.Scale(range=[0, 4000]),
          legend=alt.Legend(title='Number of Unicorns')
      ),
      alt.Color('market:N', legend=None),
      tooltip=['market','count:Q']
  ).properties(title = "In which markets the startups have higher possibility to become an unicorn?",
      width=600,
      height=500
  ).interactive()
  result.encoding.x.title = 'Successful Rate'
  result.encoding.y.title = 'Market'  
  return result

In [43]:
successfulrate(top15)

alt.Chart(...)

In [44]:
df[(df['name'].isin(unicorn_list))&(df['market']=='DataPrivacy')]

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,funding_rounds,founded_at,founded_month,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,lasting_year,Status_acquired,Status_closed,Status_operating,if_ipo,month,if_jan,found_fund_diff,fir_end_diff,founded_year_range
19808,ID.me,|Data Privacy|Identity Management|Online Ident...,DataPrivacy,16665000.0,operating,USA,DC,"Washington, D.C.",6.0,2010-01-01,2010-01,2010.0,2010-07-16,2014-04-01,140000.0,7500000.0,0.0,0.0,3000000.0,2500000.0,925000.0,2600000.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0,0,1,0,1.0,1,196.0,1355.0,"[2010, 2020)"


In [45]:
df[(df['name'].isin(unicorn_list))&(df['market']=='SharedServices')]

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,funding_rounds,founded_at,founded_month,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,lasting_year,Status_acquired,Status_closed,Status_operating,if_ipo,month,if_jan,found_fund_diff,fir_end_diff,founded_year_range
39232,Socar,|Travel|Shared Services|,SharedServices,18000000.0,operating,KOR,NaN,Busan,1.0,2011-11-10,2011-11,2011.0,2014-10-14,2014-10-14,0.0,18000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,0,1,0,11.0,0,1069.0,0.0,"[2010, 2020)"


In [46]:
successfulrate(top15f)

alt.Chart(...)

In [47]:
def Unicorn_Funding(df):
  status_fund = df.groupby('market')['funding_total_usd'].mean()
  # status_fund = df.groupby('market')['funding_total_usd'].median()
  status_fund = status_fund.reset_index().sort_values(by="funding_total_usd",ascending = False)
  status_fund.rename({'market': 'Market','funding_total_usd': 'Average_Funding'}, axis=1, inplace=True)
  
  status_fund_plt = alt.Chart(status_fund, title = 'What are the avergae funding in differnent market').mark_bar().encode(
                    alt.X('market:N', axis=alt.Axis(title=None, labels=False), sort=alt.EncodingSortField(order='descending')),
                    alt.Y('Average_Funding:Q', title = 'Average Funding in USD'),
                    column = 'Market:N',
                    color = 'Market:N',
                    tooltip = ['Market', 'Average_Funding']
                    ).properties(width=70, height=300).interactive()
  return status_fund_plt

In [48]:
Unicorn_Funding(df_unicorn)

alt.Chart(...)

In [49]:
result = alt.Chart(df_unicorn.drop('founded_year_range', 1)).mark_point().encode(
      x='funding_rounds:Q',
      y='funding_total_usd:Q',
      color='market:N',
      tooltip=['name','market','funding_total_usd','funding_rounds','founded_year']
  ).properties(title="",width=600,height=500).interactive()
result.encoding.x.title = 'Funding_Rounds'
result.encoding.y.title = 'Funding_Total' 



result

alt.Chart(...)

### Findings 2: IPO startups from 2000 to 2012

According to [Investopedia](https://www.investopedia.com/terms/i/ipo.asp), an initial public offering (IPO) refers to the process of offering shares of a private corporation to the public in a new stock issuance. Being an IPO allows a company to raise capital from public investors. Not to mention, the transition from a private to a public company can be an important time for private investors to fully realize gains from their investment as it typically includes a share premium for current private investors. Meanwhile, being an IPO also allows public investors to participate in the offering.

Based on our dataset, we flag a startup as proceeding to IPO if it gains post-IPO equity/post-IPO debt or enters the secondary market. For those startups/investors targeting to move to IPO, we have explored the features of the existing IPO startups from 2000 to 2012 in the United States. 

In [50]:
 df_usa = df_usa[(df_usa['founded_year'] >= 2000) & (df_usa['founded_year'] <= 2012)]
 df_ipo = df_usa[df_usa['if_ipo'] == 1]

- Propotion of startups proceeded to IPO in the United States from 2000 to 2012

In [51]:
start_up_num = len(df_usa)
ipo_num = sum(df_usa['if_ipo'])
ipo_rate = round(ipo_num/start_up_num*100, 2)
print('Among {} startups in the United States from 2000 to 2012, {} startups progressed along the road to becoming an IPO, which took {} % of total startups.'.\
      format(start_up_num, ipo_num, ipo_rate))

Among 17135 startups in the United States from 2000 to 2012, 92 startups progressed along the road to becoming an IPO, which took 0.54 % of total startups.


In [52]:
df_ipo_cal = df_usa.groupby(['if_ipo'])['name'].count()
df_ipo_cal = df_ipo_cal.reset_index()
df_ipo_cal['Percentage'] = round(df_ipo_cal['name']/len(df_usa)*100,2)

alt.Chart(df_ipo_cal, title= 'The startup progressed to IPO only takes 0.54% of total startups in the United States from 2000 to 2012').\
    mark_bar().encode(
    alt.X('if_ipo:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Percentage:Q', title = 'Count of Records'),
    alt.Color('if_ipo:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['if_ipo', 'Percentage']
).properties(width = 300, height = 300).interactive()


alt.Chart(...)

- Mapping the startups by state in the United States from 2000 to 2012

In [53]:
# the USA map in terms of founded year for startups progressed to IPO - more detailed view
df_map_ipo = df_ipo.groupby(['state_code','founded_year'])['name'].count()
df_map_ipo = df_map_ipo.reset_index().sort_values('founded_year', ascending=True)
df_map_ipo.rename({'name': 'Count_of_Records'}, axis=1, inplace=True)
fig = px.choropleth(df_map_ipo,
                    locations='state_code',
                    color='Count_of_Records',
                    color_continuous_scale='Rdbu',
                    hover_name='state_code',
                    animation_frame = 'founded_year',
                    locationmode='USA-states',
                    labels={'Count_of_Records':'Number of Startups'},
                    scope='usa')


fig.update_layout(
    title={'text':'Number of startups progressed to IPO by State',
           'xanchor':'center',
           'yanchor':'top',
           'x':0.5})
fig.show()

In [54]:
# the USA map for startups progressed to IPO - overview
df_map_ipo = df_ipo.groupby(['state_code'])['name'].count()
df_map_ipo= df_map_ipo.reset_index()
df_map_ipo.rename({'name': 'Count_of_Records'}, axis=1, inplace=True)
fig = px.choropleth(df_map_ipo,
                    locations='state_code',
                    color='Count_of_Records',
                    color_continuous_scale='rdbu',
                    hover_name='state_code',
                    locationmode='USA-states',
                    labels={'Count_of_Records':'Number of Startups'},
                    scope='usa')
fig.update_layout(
    title={'text':'Startups progressed along the road to becoming an IPO are mainly located in California, Massachusetts, Florida, New York, North Carolina, and Texas.',
           'xanchor':'center',
           'yanchor':'top',
           'x':0.5})
fig.show()
#present #what's the finding

Based on the visualization, those "successful startups" are mainly located in California, Massachusetts, Florida, New York, North Carolina, and Texas. Also, most startups that progressed to IPO are at the boundaries. It's not surprising that California has the most startups that proceeded to IPO, which parallels the finding that dominating startups are in California. 

Question generated from the map: 
- As a state with a cashless society in the uplands and plantation agriculture in the lowlands, what startups moved to IPO in Arkansas? Why is it unqiue compared to other startups?

- Dive deeper to look at the startups in Arkansas

In [55]:
df_ar = df_usa[df_usa['state_code'] == 'AR']

df_ar_mkt = df_ar.groupby(['market', 'if_ipo'])['name'].count()
df_ar_mkt = df_ar_mkt.reset_index().sort_values(by='name', ascending = False)
df_ar_mkt.rename({'market': 'Market', 'name': 'Count_of_Records'}, axis=1, inplace=True)

alt.Chart(df_ar_mkt, title= 'The startup progressed to IPO is in the HealthCare field, which is the only startup in that field during 2000 to 2012 in Arkansas').\
    mark_bar().encode(
    alt.X('Market:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Count_of_Records:Q', title = 'Count of Records'),
    alt.Color('if_ipo:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['Market', 'Count_of_Records']
).interactive()


alt.Chart(...)

Surprisingly, the state with a cashless society in the uplands and plantation agriculture in the lowlands doesn't contain many startups in the agriculture field but technology, medical, and health service sectors. In addition, the startup that proceeded to IPO is in the Health Care market, which is identical in all startups in Arkansas.

Based on data collected by [americashealthrankings.org](https://www.americashealthrankings.org/explore/annual/measure/Overall_a/state/AR), as of 2012, Arkansas has a high incidence of premature death, infant mortality, cardiovascular deaths, and occupational fatalities compared to the rest of the United States. In addition, Arkansas is usually ranked as one of the least healthy states due to high obesity, smoking, and sedentary lifestyle rates.

In this case, Arkansas made the most immediate progress in reducing its number of uninsured residents after the Affordable Care Act was passed. In other words, the government promoted and supported the startup in the Healthcare field. 



In [56]:
df_ar_mkt = df_ar.groupby(['market', 'if_ipo'])['funding_total_usd'].mean()
df_ar_mkt = df_ar_mkt.reset_index().sort_values(by='funding_total_usd', ascending = False)
df_ar_mkt.rename({'market': 'Market', 'funding_total_usd': 'Average_Funding'}, axis=1, inplace=True)

alt.Chart(df_ar_mkt, title= 'The field that the startup progressed to IPO in did not acquired the most funding.').\
    mark_bar().encode(
    alt.X('Market:N', sort=alt.EncodingSortField(order='descending')),
    alt.Y('Average_Funding:Q', title = 'Average_Funding'),
    alt.Color('if_ipo:N', sort=alt.EncodingSortField(order='descending')),
    tooltip = ['Market', 'Average_Funding']
).interactive()

alt.Chart(...)

It's interesting to observe that nanotechnology acquired the most average funding in Arkansas. Arkansas devotes itself to creating many new materials with various applications, including nanomedicine, nanoelectronics, biomaterials energy production. However, nanotechnology may cause issues like any new technology, including concerns about nanomaterials' toxicity and environmental impact. In our future work, we would love to collect the environmental data in Arkansas to explore the positive or negative effects of having more startups in nanotechnology fields.

- Observing the startups proceeded to IPO by market in the United States from 2000 to 2012

In [57]:
df_ipo_mkt = df_ipo.groupby(['market'])['name'].count()
df_ipo_mkt = df_ipo_mkt.reset_index().sort_values(by = 'name', ascending = False)[:20]
df_ipo_mkt.rename({'market': 'Market', 'name': 'Count_of_Records'}, axis=1, inplace=True)

ipo_mkt_plt = alt.Chart(df_ipo_mkt, title = 'Number of startups progressed to IPO in the Health Care and Biotechnology field greatly exceeds Software.').mark_bar().encode(
              alt.X('Market:N', sort=alt.EncodingSortField(order='descending')),
              alt.Y('Count_of_Records:Q', title = 'Count of Records'),
              alt.Color('Market:N', sort=alt.EncodingSortField(order='descending')),
              tooltip = ['Market', 'Count_of_Records']
              ).properties(width=800, height=300).interactive()

ipo_mkt_plt

alt.Chart(...)

It's interesting to observe that among all startups that progressed to IPO, startups in Health Care and Biotechnology dominate the rest of the markets. Frankly, the number of startups that moved to IPO in software is tiny regarding the significant total number of startups in the software market. 

### Findings 3: Most startups begin their business in January

In [58]:
def Number_Month_Con(start, end, country, df):
  '''
  Plot a line chart to display the month distribution for a country to start their business within given time period
  '''
  df_month = df[(df['founded_year'] >= start) & (df['founded_year'] <= end) & (df['country_code'] == country)]
  df_month = df_month.groupby(['founded_year', 'month'])['name'].count()
  df_month = df_month.reset_index()
  df_month.rename({'founded_year': 'Founded_Year', 'month': 'Month', 'name': 'Count_of_Records'}, axis=1, inplace=True)
  

  mth_plt = alt.Chart(df_month, title = 'A great amount of startups in the United States tend to start their business in January {} to {} in {}'.\
                      format(start, end, country)).mark_line().encode(
                alt.X('Month:N', title = 'Founded Month'),
                alt.Y('Count_of_Records:Q', title = 'Count of Records'),
                color = 'Founded_Year:N',
                tooltip = ['Month', 'Count_of_Records', 'Founded_Year']
                ).properties(width=800, height=500).interactive()
  return mth_plt

In [59]:
Number_Month_Con(2000, 2012, 'USA', df)
#present

alt.Chart(...)

From this visualization, it's interested to notice that most companies in the United States tend to start their business in January.
We research on the potential reasons for startups to start their business in January as [following](https://www.allbusiness.com/new-year-best-time-to-start-a-business-134520-1.html):
1. January is a **fresh start of a new year**.
2. **Bookkeeping, the recording of financial transactions, is much easier for companies starting their business in January.** In general, a company must file tax returns starting from the year that it is established. Startup have chances to choose to start the business late in the year and file the business tax return right away, or to choose a January start date for their business entity and file their first tax return the following year.
3. State corporation officials and staff might have **a massive backlog of business filings** due to a large amount of entity formation documentation entered every year. In other words, entrepreneurs might face unusually long wait times for their businesses to be officially established. 

Both of us wonder if the performance(total funding, how far do they go in funding rounds, reach ipo) of those companies started in January is better than companies started in other months?

- Performance of startups established in January compared to startups established in the rest of the year from 2000 to 2012.

In [60]:
# average funding for companies strated in Jan and not in Jan in terms of the funding rounds
jan_fund = df_usa.groupby(['if_jan'])['funding_total_usd'].mean()
jan_fund = jan_fund.reset_index()
jan_fund.rename({'if_jan' : 'January', 'funding_total_usd': 'Average_Funding'}, axis=1, inplace=True)

jan_fund_plt = alt.Chart(jan_fund, 
                         title = 'The average funding that gives to startups stating from January is greater than others').mark_bar().encode(
                         alt.X('January:N'),
                         alt.Y('Average_Funding:Q', title = 'Average_Funding'),
                         color = 'January:N',
                         tooltip = ['January', 'Average_Funding']
                         ).properties(width=600, height=300).interactive()

jan_fund_plt

alt.Chart(...)

In [61]:
# number of companies started in Jan and not in Jan in terms of company status
jan_status = df_usa.groupby(['if_jan', 'status'])['name'].count()
jan_status = jan_status.reset_index()
jan_status.rename({'if_jan' : 'January', 'status': 'Status', 'name': 'Count_of_Records'}, axis=1, inplace=True)

jan_status_plt = alt.Chart(jan_status, 
                         title = 'Number of opearing and acquired startups that is established in January are much higher than those established in other months.').mark_bar().encode(
                         alt.X('January:N', title = 'If January'),
                         alt.Y('Count_of_Records:Q', title = 'Count of Records'),
                         color = 'Status:N',
                         column = 'Status:N',
                         tooltip = ['January', 'Status', 'Count_of_Records']
                         ).properties(width=300, height=200).interactive()

jan_status_plt
#present

alt.Chart(...)

In [62]:
# number of companies started in Jan and not in Jan in terms of if the company progressed to IPO
jan_ipo = df_ipo.groupby(['if_jan', 'if_ipo'])['name'].count()
jan_ipo = jan_ipo.reset_index()
jan_ipo['Percentage'] = round(jan_ipo['name']/len(df_ipo)*100,2)
jan_ipo.rename({'if_jan' : 'January', 'name': 'Count_of_Records'}, axis=1, inplace=True)

jan_ipo_plt = alt.Chart(jan_ipo, 
                         title = 'Among all startups progressed to IPO, 77% of them were founded in January.').mark_bar().encode(
                         alt.X('January:N', title = 'If January'),
                         alt.Y('Percentage:Q', title = 'Being IPO Pecentage'),
                         color = 'January:N',
                         tooltip = ['January',  'Percentage']
                         ).properties(width=500, height=300).interactive()

jan_ipo

,January,if_ipo,Count_of_Records,Percentage
0,0,1,21,22.83
1,1,1,71,77.17


### Future Work

#### Potential reasons that cause the drop of startups worldwide

In [63]:
df_year = df[(df['founded_year'] >= 1978)]
df_year = df_year.groupby(['founded_year'])['name'].count()
df_year = df_year.reset_index()
df_year.rename({'founded_year': 'Founded_Year', 'name': 'Count_of_Records'}, axis=1, inplace=True)


year_plt = alt.Chart(df_year, title = 'Number of startups reaches the peak in 2012 worldwide').mark_line().encode(
           alt.X('Founded_Year:N', title = 'Founded Year'),
           alt.Y('Count_of_Records:Q', title = 'Count of Records'),
           tooltip = ['Founded_Year', 'Count_of_Records']
           ).properties(width=800, height=300).interactive()
year_plt

alt.Chart(...)

Based on the visualization above,  we find two small drops from 1902 to 2011, occurring in 2000 and 2007. However, the number of startups has started to drop dramatically since 2012 and has continued to decline. 

Due to the time and data limitation, we cannot explore the potential reasons that cause the massive reduction of startups. However, we come up with some hypotheses that can help us to research more on this topic in the future:

- **The shrinking of population growth affects the birth of new startups.** According to [Robert Litan](https://www.inc.com/magazine/201505/leigh-buchanan/the-vanishing-startups-in-decline.html), an economist at Brookings, "On the supply side, if you have fewer people, fewer companies are being formed. On the demand side, a slowing population means less demand for new products." In addition, it's hard to see a breakout of new startups in aging societies like Japan.
- **The established companies may take advantage of being pioneers and experts in a specific field. Once those established companies grow more prominent, the startups are less likely to sprout.** Based on the data collected by the Federal Reserve Bank of Cleveland, around 80% of arising companies were "new establishments" in 1978. The rest of them were new locations for established companies. Yet, the rate decreases to 60% these days. Customers are likely to visit or purchase the products or services provided in those well-known brands rather than an arising startup. 



#### The impact of Obama's "Startup America" policy



[**"Startup America"**](https://obamawhitehouse.archives.gov/startup-america-fact-sheet) is a White House initiative to celebrate, inspire, and accelerate high-growth entrepreneurship throughout the nation. 

The main commitments in this startegy includes:
- Expand access to capital for high-growth startups throughout the country
- Expand entrepreneurship education and mentorship programs that empower more Americans not just to get a job, but to create jobs
- Strengthen commercialization of the about $148 billion in annual federally-funded research and development, which can generate innovative startups and entirely new industries
- Identify and remove unnecessary barriers to high-growth startups
- Expand collaborations between large companies and startups.

Not to mention, this policy commits $1 billion to those funds that invest growth capital in companies located in underserved communities such as clean technology.

In [64]:
# the startup growth rate from 1978 to 2014 in the United States

df_year_usa = df[(df['founded_year'] >= 1978) & (df['country_code'] == 'USA')]
df_year_usa = df_year_usa.groupby(['founded_year'])['name'].count()
df_year_usa = df_year_usa.reset_index()
df_year_usa['Growth_Rate'] = df_year_usa['name'].pct_change().mul(100).round(2)
df_year_usa.rename({'founded_year': 'Founded_Year', 'name': 'Count_of_Records'}, axis=1, inplace=True)


year_plt = alt.Chart(df_year_usa[1:], 
           title = 'The growth rate of startups fluctuated in the United States from 1978 to 2012 and has dropped dramatically since 2012.').mark_line().encode(
           alt.X('Founded_Year:N', title = 'Founded Year'),
           alt.Y('Growth_Rate:Q', title = 'Growth Rate'),
           tooltip = ['Founded_Year', 'Growth_Rate', 'Count_of_Records']
           ).interactive()
year_plt

alt.Chart(...)

Based on the visualization, we observe that the growth rate of startups fluctuated in the United States from 1978 to 2012 and has dropped dramatically since 2012. We wonder why the growth rate continued to fall even if the United States had launched the "Startup America" strategy. To explore the effect of "Startup America," we would love to collect data after 2014 to view the startups in the United States completely.

In [65]:
df_year_usa = df[(df['founded_year'] >= 2000) & (df['country_code'] == 'USA')]
df_year = df_year_usa[(df_year_usa['market'] == 'CleanTechnology')]


df_year = df_year.groupby(['founded_year'])['name'].count()
df_year = df_year.reset_index()
df_year.rename({'founded_year': 'Founded_Year', 'name': 'Count_of_Records'}, axis=1, inplace=True)

year_plt = alt.Chart(df_year, title = 'Number of startups in the Clean Technology market drop since 2011 in the United States').mark_line().encode(
           alt.X('Founded_Year:N', title = 'Founded Year'),
           alt.Y('Count_of_Records:Q', title = 'Count of Records'),
           tooltip = ['Founded_Year', 'Count_of_Records']
           ).properties(width=800, height=300).interactive()
year_plt


alt.Chart(...)

We don't observe the positive impact of "Startup America," specifically in the clean technology market. The reason may seem because of the delay effect or the data incompleteness. In this case, we would love to collect more data to explore the impact in the future. 